# 🎓 Aula 1: A Nova Era dos Agentes Cognitivos

## Performance – IA para Devs | Desenvolvimento e Orquestração de Agentes de IA para o Setor Financeiro

---

### 📋 Objetivos desta Aula

Ao final desta aula, você será capaz de:

1. **Compreender a evolução** de Chains lineares para Agentes autônomos
2. **Dominar os parâmetros de modelos LLM** (Temperature, Top-P, Top-K, Tokens) e entender precificação
3. **Construir a anatomia completa de um Agente**: Cérebro (LLM) + Memória + Ferramentas
4. **Implementar o padrão ReAct** (Reasoning + Acting)
5. **Desenvolver o projeto "O Analista Júnior"**: Um agente financeiro com Custom Tools

---

### 🛠️ Stack Tecnológico

- **Python 3.10+**
- **LangChain Core** - Framework para desenvolvimento de aplicações com LLMs
- **Pydantic** - Validação de dados e definição de schemas
- **OpenAI API** - Modelo de linguagem (ou alternativas)

---

## 🚀 Parte 0: Configuração do Ambiente

Antes de começarmos, precisamos instalar e configurar todas as dependências necessárias.

In [2]:
# Instalação das dependências necessárias
# Execute esta célula apenas uma vez

%pip install langchain langchain-openai langchain-core python-dotenv pydantic --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Configuração das variáveis de ambiente
# IMPORTANTE: Substitua pela sua chave de API da OpenAI

import os
from dotenv import load_dotenv

# Tenta carregar de um arquivo .env (se existir)
load_dotenv()

# Se não tiver um arquivo .env, defina diretamente aqui:
# ⚠️ NUNCA compartilhe sua chave de API!
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "sua-chave-api-aqui"  # Substitua pela sua chave

print("✅ Ambiente configurado com sucesso!")
print(f"🔑 API Key configurada: {'Sim' if os.getenv('OPENAI_API_KEY') and os.getenv('OPENAI_API_KEY') != 'sua-chave-api-aqui' else 'Não - Configure sua chave!'}")

✅ Ambiente configurado com sucesso!
🔑 API Key configurada: Sim


---

## 📚 Parte 1: A Evolução - De Chains Lineares para Agentes Autônomos

### 1.1 O Paradigma Tradicional: Chains (Correntes)

No início da era LLM, as aplicações seguiam um modelo **linear e determinístico**:

```
Input → Prompt Template → LLM → Output Parser → Resultado Final
```

**Características das Chains:**
- ✅ Previsíveis e fáceis de debugar
- ✅ Baixo custo computacional
- ❌ Inflexíveis - não se adaptam a situações inesperadas
- ❌ Não conseguem "pensar" sobre qual caminho seguir

### 1.2 O Novo Paradigma: Agentes Autônomos

Agentes representam uma **mudança fundamental**: em vez de seguir um caminho fixo, eles **decidem dinamicamente** o que fazer.

```
Input → Agente (LLM + Raciocínio) → [Decisão] → Ferramenta A ou B ou C → [Nova Decisão] → ...
```

**Características dos Agentes:**
- ✅ Autônomos - tomam decisões em tempo real
- ✅ Adaptáveis - mudam de estratégia conforme necessário
- ✅ Podem usar múltiplas ferramentas
- ❌ Menos previsíveis
- ❌ Potencialmente mais custosos (mais chamadas à API)

---

### 💻 Demonstração: Chain Linear vs Agente

In [4]:
# EXEMPLO 1: Chain Linear Tradicional
# Um fluxo fixo e previsível

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Garante que a chave da OpenAI está configurada
api_key = os.getenv("OPENAI_API_KEY")
if not api_key or api_key == "sua-chave-api-aqui":
    raise ValueError(
        "Configuração ausente ou inválida: defina OPENAI_API_KEY antes de executar. "
        "Ex.: os.environ['OPENAI_API_KEY'] = 'sk-...' ou configure no .env"
    )

# Inicializa o modelo
llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)

# Define o template do prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um analista financeiro. Responda de forma concisa."),
    ("human", "{pergunta}")
])

# Cria a chain: Prompt → LLM → Parser
chain_linear = prompt | llm | StrOutputParser()

# Executa a chain
resultado = chain_linear.invoke({"pergunta": "O que é diversificação de carteira?"})

print("🔗 CHAIN LINEAR - Fluxo Fixo")
print("=" * 50)
print(resultado)

🔗 CHAIN LINEAR - Fluxo Fixo
Diversificação de carteira é a estratégia de investir em diferentes ativos financeiros, como ações, títulos, imóveis e commodities, para reduzir o risco global do portfólio. Ao espalhar os investimentos, a ideia é que a performance negativa de um ativo possa ser compensada pela performance positiva de outro, melhorando a estabilidade e potencial de retorno do portfólio.


In [5]:
# EXEMPLO 2: Limitação da Chain Linear
# O que acontece quando precisamos de dados em tempo real?

resultado_limitado = chain_linear.invoke({
    "pergunta": "Qual é a cotação atual do dólar e devo comprar agora?"
})

print("🔗 CHAIN LINEAR - Tentando responder sobre dados em tempo real")
print("=" * 50)
print(resultado_limitado)
print("\n⚠️ PROBLEMA: A chain não tem acesso a dados externos!")
print("   Ela só pode usar o conhecimento do modelo, que tem um corte temporal.")

🔗 CHAIN LINEAR - Tentando responder sobre dados em tempo real
Desculpe, mas não consigo fornecer cotações em tempo real. Recomendo verificar fontes financeiras atualizadas. Quanto a comprar, considere fatores como sua necessidade de câmbio, tendências do mercado e a cotação atual em relação à sua expectativa de valorização ou desvalorização futura. Avalie também suas condições financeiras antes de decidir.

⚠️ PROBLEMA: A chain não tem acesso a dados externos!
   Ela só pode usar o conhecimento do modelo, que tem um corte temporal.


### 🎯 Insight Fundamental

A chain linear **não consegue** buscar informações externas. Ela está limitada ao conhecimento estático do modelo.

**Agentes resolvem isso** porque podem:
1. Reconhecer que precisam de informação externa
2. Escolher a ferramenta certa para buscar essa informação
3. Usar a informação para formular uma resposta

---

## 📚 Parte 2: Parâmetros do Modelo LLM

Antes de construir agentes, é crucial entender como **controlar o comportamento** do LLM.

### 2.1 Temperature (Temperatura)

**O que é:** Controla a "criatividade" ou aleatoriedade das respostas.

| Valor | Comportamento | Uso Recomendado |
|-------|--------------|------------------|
| 0.0 | Determinístico, sempre a mesma resposta | Análises financeiras, dados precisos |
| 0.3-0.5 | Levemente variado, mas consistente | Chatbots profissionais |
| 0.7-1.0 | Criativo, respostas variadas | Brainstorming, conteúdo criativo |
| > 1.0 | Muito aleatório, pode ser incoerente | Raramente usado |

### 2.2 Top-P (Nucleus Sampling)

**O que é:** Limita a seleção de tokens aos mais prováveis que somam até P%.

- `top_p=0.1`: Considera apenas os tokens que representam 10% da probabilidade total (muito conservador)
- `top_p=0.9`: Considera tokens até 90% da probabilidade (mais diverso)
- `top_p=1.0`: Considera todos os tokens (padrão)

### 2.3 Top-K

**O que é:** Limita a seleção aos K tokens mais prováveis.

- `top_k=1`: Sempre escolhe o token mais provável (greedy)
- `top_k=40`: Considera os 40 tokens mais prováveis (padrão em alguns modelos)
- `top_k=100`: Mais diversidade

### 2.4 Tokens e Context Window

**Token:** Unidade básica de texto (~4 caracteres em inglês, ~3 em português)

**Context Window:** Quantidade máxima de tokens que o modelo pode processar de uma vez.

| Modelo | Context Window | Custo Input (1M tokens) | Custo Output (1M tokens) |
|--------|----------------|------------------------|-------------------------|
| GPT-4o-mini | 128K | $0.15 | $0.60 |
| GPT-4o | 128K | $2.50 | $10.00 |
| GPT-4 Turbo | 128K | $10.00 | $30.00 |

---

### 💻 Demonstração Prática dos Parâmetros

In [6]:
# Demonstração: Efeito da Temperature

from langchain_openai import ChatOpenAI

pergunta = "Sugira um nome criativo para um fundo de investimento focado em tecnologia."

print("🌡️ DEMONSTRAÇÃO: Efeito da Temperature")
print("=" * 60)

# Temperature = 0 (Determinístico)
llm_temp_0 = ChatOpenAI(model="gpt-4o-mini", temperature=1)
print("\n📊 Temperature = 0 (Determinístico):")
for i in range(3):
    resposta = llm_temp_0.invoke(pergunta)
    print(f"   Tentativa {i+1}: {resposta.content}")

🌡️ DEMONSTRAÇÃO: Efeito da Temperature

📊 Temperature = 0 (Determinístico):
   Tentativa 1: Claro! Que tal o nome "TechNova Capital"? Esse nome sugere inovação e um foco nas novas tecnologias, transmitindo a ideia de que o fundo está sempre à frente, investindo em soluções do futuro.
   Tentativa 2: Claro! Que tal o nome "InovaTech Capital"? Ele transmite a ideia de inovação e tecnologia, além de ter um tom profissional e confiável.
   Tentativa 3: Claro! Que tal "TechVenture Capital"? Esse nome transmite a ideia de aventura e inovação no mundo da tecnologia, ao mesmo tempo que remete ao conceito de capital de risco. Outras sugestões podem incluir:

1. "InovaTech Fund"
2. "FutureTech Insights"
3. "Digital Frontier Investments"
4. "TechNova Ventures"
5. "TechPulse Capital"
6. "NextGen Tech Fund"
7. "SynergyTech Investments"
8. "QuantumLeap Fund"

Espero que uma dessas sugestões ressoe com a sua visão!


In [7]:
# Temperature = 1.0 (Criativo)
llm_temp_1 = ChatOpenAI(model="gpt-4o-mini", temperature=1.0)
print("\n🎨 Temperature = 1.0 (Criativo):")
for i in range(3):
    resposta = llm_temp_1.invoke(pergunta)
    print(f"   Tentativa {i+1}: {resposta.content}")

print("\n💡 Observe: Com temperature=0, as respostas são idênticas.")
print("   Com temperature=1.0, cada resposta é diferente!")


🎨 Temperature = 1.0 (Criativo):
   Tentativa 1: Claro! Que tal o nome "InovaTech Capital"? Esse nome combina a ideia de inovação com o foco em tecnologia, refletindo um compromisso com o futuro e o crescimento no setor.
   Tentativa 2: Claro! Que tal "TechNova Capital"? Esse nome transmite a ideia de inovação e novas tecnologias, além de remeter a um futuro promissor.
   Tentativa 3: "TechNova Capital"

💡 Observe: Com temperature=0, as respostas são idênticas.
   Com temperature=1.0, cada resposta é diferente!


In [8]:
# Demonstração: Contagem de Tokens e Estimativa de Custo

import tiktoken

def contar_tokens(texto: str, modelo: str = "gpt-4o-mini") -> int:
    """Conta o número de tokens em um texto."""
    try:
        encoding = tiktoken.encoding_for_model(modelo)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(texto))

def estimar_custo(tokens_input: int, tokens_output: int, modelo: str = "gpt-4o-mini") -> float:
    """Estima o custo em USD baseado nos tokens."""
    precos = {
        "gpt-4o-mini": {"input": 0.15 / 1_000_000, "output": 0.60 / 1_000_000},
        "gpt-4o": {"input": 2.50 / 1_000_000, "output": 10.00 / 1_000_000},
        "gpt-4-turbo": {"input": 10.00 / 1_000_000, "output": 30.00 / 1_000_000},
    }
    preco = precos.get(modelo, precos["gpt-4o-mini"])
    return (tokens_input * preco["input"]) + (tokens_output * preco["output"])

# Exemplo prático
texto_exemplo = """
Análise de Crédito - Cliente: João Silva
Renda mensal: R$ 15.000,00
Comprometimento atual: 25%
Score de crédito: 780
Histórico: Sem atrasos nos últimos 24 meses
Solicitação: Empréstimo de R$ 50.000,00 em 36 parcelas
"""

tokens = contar_tokens(texto_exemplo)
tokens_resposta_estimados = 200  # Estimativa da resposta

print("📊 ANÁLISE DE TOKENS E CUSTOS")
print("=" * 50)
print(f"\nTexto de entrada:\n{texto_exemplo}")
print(f"\n🔢 Tokens no texto: {tokens}")
print(f"📝 Tokens estimados na resposta: {tokens_resposta_estimados}")
print(f"\n💰 Custo estimado por modelo:")

for modelo in ["gpt-4o-mini", "gpt-4o", "gpt-4-turbo"]:
    custo = estimar_custo(tokens, tokens_resposta_estimados, modelo)
    custo_1000_requisicoes = custo * 1000
    print(f"   {modelo}: ${custo:.6f} por requisição | ${custo_1000_requisicoes:.2f} para 1.000 requisições")

📊 ANÁLISE DE TOKENS E CUSTOS

Texto de entrada:

Análise de Crédito - Cliente: João Silva
Renda mensal: R$ 15.000,00
Comprometimento atual: 25%
Score de crédito: 780
Histórico: Sem atrasos nos últimos 24 meses
Solicitação: Empréstimo de R$ 50.000,00 em 36 parcelas


🔢 Tokens no texto: 73
📝 Tokens estimados na resposta: 200

💰 Custo estimado por modelo:
   gpt-4o-mini: $0.000131 por requisição | $0.13 para 1.000 requisições
   gpt-4o: $0.002182 por requisição | $2.18 para 1.000 requisições
   gpt-4-turbo: $0.006730 por requisição | $6.73 para 1.000 requisições


### 🎯 Boas Práticas para o Setor Financeiro

| Cenário | Temperature | Top-P | Modelo Recomendado |
|---------|-------------|-------|--------------------|
| Análise de risco | 0 | 1.0 | GPT-4o (precisão) |
| Extração de dados | 0 | 1.0 | GPT-4o-mini (custo) |
| Chatbot de atendimento | 0.3 | 0.9 | GPT-4o-mini |
| Geração de relatórios | 0.2 | 0.95 | GPT-4o |

---

## 📚 Parte 3: Anatomia de um Agente Cognitivo

Um agente é composto por **três componentes fundamentais**:

### 🧠 3.1 Cérebro (LLM)
O modelo de linguagem que processa informações e toma decisões.

### 💾 3.2 Memória
Capacidade de lembrar interações anteriores para manter contexto.

**Tipos de Memória:**
- **Buffer Memory:** Armazena as últimas N mensagens
- **Summary Memory:** Resume conversas longas
- **Vector Memory:** Busca semântica em histórico longo

### 🔧 3.3 Ferramentas (Tools)
Funções externas que o agente pode invocar para realizar ações.

```
┌─────────────────────────────────────────────────────────┐
│                    AGENTE COGNITIVO                      │
├─────────────────────────────────────────────────────────┤
│                                                          │
│   ┌──────────┐     ┌──────────┐     ┌──────────┐        │
│   │  🧠 LLM  │ ←→  │ 💾Memória│ ←→  │🔧 Tools │        │
│   │ (Cérebro)│     │          │     │          │        │
│   └──────────┘     └──────────┘     └──────────┘        │
│        ↑                                  │              │
│        │              ┌───────────────────┘              │
│        │              ↓                                  │
│   ┌──────────┐   ┌─────────────────────────────┐        │
│   │  Input   │   │  APIs, Bancos de Dados,     │        │
│   │ (Humano) │   │  Calculadoras, Web Search   │        │
│   └──────────┘   └─────────────────────────────┘        │
│                                                          │
└─────────────────────────────────────────────────────────┘
```

---

### 💻 Construindo os Componentes

In [9]:
# COMPONENTE 1: O Cérebro (LLM)

from langchain_openai import ChatOpenAI

# Configuração otimizada para o setor financeiro
cerebro = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,  # Determinístico para decisões financeiras
    max_tokens=1000,  # Limite de tokens na resposta # type: ignore
)

print("🧠 CÉREBRO configurado!")
print(f"   Modelo: gpt-4o-mini")
print(f"   Temperature: 0 (determinístico)")
print(f"   Max Tokens: 1000")

🧠 CÉREBRO configurado!
   Modelo: gpt-4o-mini
   Temperature: 0 (determinístico)
   Max Tokens: 1000


In [10]:
# COMPONENTE 2: A Memória

from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Dicionário para armazenar históricos por sessão
historicos: dict[str, InMemoryChatMessageHistory] = {}

def obter_historico(session_id: str) -> InMemoryChatMessageHistory:
    """Retorna o histórico de uma sessão específica."""
    if session_id not in historicos:
        historicos[session_id] = InMemoryChatMessageHistory()
    return historicos[session_id]

print("💾 MEMÓRIA configurada!")
print("   Tipo: InMemoryChatMessageHistory")
print("   Função: Armazena conversas por sessão")

💾 MEMÓRIA configurada!
   Tipo: InMemoryChatMessageHistory
   Função: Armazena conversas por sessão


In [11]:
# COMPONENTE 3: As Ferramentas (Tools)
# Vamos criar ferramentas específicas para o setor financeiro

from langchain_core.tools import tool
from pydantic import BaseModel, Field
from typing import Optional
import random
from datetime import datetime

# ===== FERRAMENTA 1: Consulta de Cotação de Ações =====
class CotacaoInput(BaseModel):
    """Input para consulta de cotação de ações."""
    simbolo: str = Field(description="Símbolo da ação (ex: PETR4, VALE3, ITUB4)")

@tool(args_schema=CotacaoInput)
def consultar_cotacao(simbolo: str) -> dict:
    """
    Consulta a cotação atual de uma ação na B3.
    Use esta ferramenta quando precisar saber o preço atual de uma ação brasileira.
    """
    # Simulação de dados de mercado (em produção, conectaria a uma API real)
    acoes_simuladas = {
        "PETR4": {"nome": "Petrobras PN", "preco_base": 38.50, "variacao_max": 2.0},
        "VALE3": {"nome": "Vale ON", "preco_base": 62.30, "variacao_max": 3.0},
        "ITUB4": {"nome": "Itaú Unibanco PN", "preco_base": 32.80, "variacao_max": 1.5},
        "BBDC4": {"nome": "Bradesco PN", "preco_base": 12.45, "variacao_max": 0.8},
        "ABEV3": {"nome": "Ambev ON", "preco_base": 11.20, "variacao_max": 0.5},
        "WEGE3": {"nome": "WEG ON", "preco_base": 52.60, "variacao_max": 2.5},
        "MGLU3": {"nome": "Magazine Luiza ON", "preco_base": 2.15, "variacao_max": 0.3},
        "B3SA3": {"nome": "B3 ON", "preco_base": 10.85, "variacao_max": 0.6},
    }
    
    simbolo_upper = simbolo.upper()
    
    if simbolo_upper not in acoes_simuladas:
        return {
            "erro": f"Ação '{simbolo}' não encontrada.",
            "acoes_disponiveis": list(acoes_simuladas.keys())
        }
    
    dados = acoes_simuladas[simbolo_upper]
    variacao = random.uniform(-dados["variacao_max"], dados["variacao_max"])
    preco_atual = round(dados["preco_base"] + variacao, 2)
    variacao_percentual = round((variacao / dados["preco_base"]) * 100, 2)
    
    return {
        "simbolo": simbolo_upper,
        "nome": dados["nome"],
        "preco_atual": preco_atual,
        "moeda": "BRL",
        "variacao_dia": variacao_percentual,
        "horario_consulta": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "status": "mercado_aberto" if 10 <= datetime.now().hour < 17 else "mercado_fechado"
    }

print("🔧 FERRAMENTA 1: consultar_cotacao")
print(f"   Descrição: {consultar_cotacao.description}")

🔧 FERRAMENTA 1: consultar_cotacao
   Descrição: Consulta a cotação atual de uma ação na B3.
Use esta ferramenta quando precisar saber o preço atual de uma ação brasileira.


In [12]:
# ===== FERRAMENTA 2: Consulta de Indicadores do Cliente =====

class ClienteInput(BaseModel):
    """Input para consulta de dados do cliente."""
    cpf: str = Field(description="CPF do cliente (apenas números)")

@tool(args_schema=ClienteInput)
def consultar_cliente(cpf: str) -> dict:
    """
    Consulta os dados financeiros de um cliente no sistema interno.
    Use esta ferramenta para obter informações sobre score de crédito, renda e histórico.
    """
    # Simulação de base de clientes
    clientes_simulados = {
        "12345678900": {
            "nome": "Maria Silva",
            "score_credito": 820,
            "renda_mensal": 15000.00,
            "comprometimento_renda": 0.25,
            "tempo_conta_anos": 8,
            "historico_atrasos": 0,
            "perfil_investidor": "Moderado"
        },
        "98765432100": {
            "nome": "João Santos",
            "score_credito": 650,
            "renda_mensal": 5500.00,
            "comprometimento_renda": 0.45,
            "tempo_conta_anos": 2,
            "historico_atrasos": 3,
            "perfil_investidor": "Conservador"
        },
        "11122233344": {
            "nome": "Ana Oliveira",
            "score_credito": 750,
            "renda_mensal": 25000.00,
            "comprometimento_renda": 0.15,
            "tempo_conta_anos": 12,
            "historico_atrasos": 1,
            "perfil_investidor": "Arrojado"
        },
    }
    
    cpf_limpo = cpf.replace(".", "").replace("-", "").replace(" ", "")
    
    if cpf_limpo not in clientes_simulados:
        return {
            "erro": f"Cliente com CPF '{cpf}' não encontrado no sistema.",
            "sugestao": "Verifique se o CPF está correto ou se o cliente está cadastrado."
        }
    
    dados = clientes_simulados[cpf_limpo]
    dados["cpf"] = cpf_limpo
    dados["data_consulta"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    return dados

print("🔧 FERRAMENTA 2: consultar_cliente")
print(f"   Descrição: {consultar_cliente.description}")

🔧 FERRAMENTA 2: consultar_cliente
   Descrição: Consulta os dados financeiros de um cliente no sistema interno.
Use esta ferramenta para obter informações sobre score de crédito, renda e histórico.


In [13]:
# ===== FERRAMENTA 3: Calculadora de Risco =====

class RiscoInput(BaseModel):
    """Input para cálculo de risco de operação."""
    valor_operacao: float = Field(description="Valor da operação em reais")
    prazo_meses: int = Field(description="Prazo da operação em meses")
    score_cliente: int = Field(description="Score de crédito do cliente (0-1000)")
    comprometimento_atual: float = Field(description="Percentual atual de comprometimento da renda (0.0 a 1.0)")

@tool(args_schema=RiscoInput)
def calcular_risco(valor_operacao: float, prazo_meses: int, score_cliente: int, comprometimento_atual: float) -> dict:
    """
    Calcula o risco de uma operação de crédito baseado em múltiplos fatores.
    Retorna uma classificação de risco e recomendação.
    """
    # Cálculo do score de risco (0-100, onde 100 é maior risco)
    risco_score = 0
    fatores = []
    
    # Fator 1: Score de crédito
    if score_cliente >= 800:
        risco_score += 5
        fatores.append("Score excelente: +5 pontos de risco")
    elif score_cliente >= 700:
        risco_score += 15
        fatores.append("Score bom: +15 pontos de risco")
    elif score_cliente >= 600:
        risco_score += 35
        fatores.append("Score médio: +35 pontos de risco")
    else:
        risco_score += 50
        fatores.append("Score baixo: +50 pontos de risco")
    
    # Fator 2: Comprometimento de renda
    if comprometimento_atual <= 0.3:
        risco_score += 5
        fatores.append("Comprometimento baixo (<=30%): +5 pontos de risco")
    elif comprometimento_atual <= 0.5:
        risco_score += 20
        fatores.append("Comprometimento médio (30-50%): +20 pontos de risco")
    else:
        risco_score += 40
        fatores.append("Comprometimento alto (>50%): +40 pontos de risco")
    
    # Fator 3: Prazo
    if prazo_meses <= 12:
        risco_score += 5
        fatores.append("Prazo curto (<=12 meses): +5 pontos de risco")
    elif prazo_meses <= 36:
        risco_score += 10
        fatores.append("Prazo médio (12-36 meses): +10 pontos de risco")
    else:
        risco_score += 20
        fatores.append("Prazo longo (>36 meses): +20 pontos de risco")
    
    # Classificação final
    if risco_score <= 25:
        classificacao = "BAIXO"
        recomendacao = "Aprovar - Operação de baixo risco"
        taxa_sugerida = 1.2
    elif risco_score <= 50:
        classificacao = "MÉDIO"
        recomendacao = "Aprovar com cautela - Considerar garantias adicionais"
        taxa_sugerida = 1.8
    elif risco_score <= 75:
        classificacao = "ALTO"
        recomendacao = "Revisar - Necessita aprovação de comitê"
        taxa_sugerida = 2.5
    else:
        classificacao = "MUITO ALTO"
        recomendacao = "Não aprovar - Risco excessivo"
        taxa_sugerida = None
    
    return {
        "score_risco": risco_score,
        "classificacao": classificacao,
        "recomendacao": recomendacao,
        "taxa_juros_mensal_sugerida": taxa_sugerida,
        "valor_operacao": valor_operacao,
        "prazo_meses": prazo_meses,
        "fatores_analisados": fatores
    }

print("🔧 FERRAMENTA 3: calcular_risco")
print(f"   Descrição: {calcular_risco.description}")

🔧 FERRAMENTA 3: calcular_risco
   Descrição: Calcula o risco de uma operação de crédito baseado em múltiplos fatores.
Retorna uma classificação de risco e recomendação.


In [14]:
# ===== FERRAMENTA 4: Conversor de Moedas =====

class ConversaoInput(BaseModel):
    """Input para conversão de moedas."""
    valor: float = Field(description="Valor a ser convertido")
    moeda_origem: str = Field(description="Código da moeda de origem (USD, EUR, BRL)")
    moeda_destino: str = Field(description="Código da moeda de destino (USD, EUR, BRL)")

@tool(args_schema=ConversaoInput)
def converter_moeda(valor: float, moeda_origem: str, moeda_destino: str) -> dict:
    """
    Converte valores entre moedas usando cotações atualizadas.
    Moedas suportadas: USD (Dólar), EUR (Euro), BRL (Real).
    """
    # Cotações simuladas (em produção, usar API de câmbio)
    cotacoes_para_brl = {
        "USD": 5.45 + random.uniform(-0.1, 0.1),  # Dólar
        "EUR": 5.95 + random.uniform(-0.1, 0.1),  # Euro
        "BRL": 1.0,  # Real
    }
    
    moeda_origem = moeda_origem.upper()
    moeda_destino = moeda_destino.upper()
    
    if moeda_origem not in cotacoes_para_brl or moeda_destino not in cotacoes_para_brl:
        return {
            "erro": "Moeda não suportada",
            "moedas_disponiveis": list(cotacoes_para_brl.keys())
        }
    
    # Converte para BRL primeiro, depois para moeda destino
    valor_em_brl = valor * cotacoes_para_brl[moeda_origem]
    valor_convertido = valor_em_brl / cotacoes_para_brl[moeda_destino]
    
    taxa_conversao = cotacoes_para_brl[moeda_origem] / cotacoes_para_brl[moeda_destino]
    
    return {
        "valor_original": valor,
        "moeda_origem": moeda_origem,
        "valor_convertido": round(valor_convertido, 2),
        "moeda_destino": moeda_destino,
        "taxa_conversao": round(taxa_conversao, 4),
        "cotacao_dolar_brl": round(cotacoes_para_brl["USD"], 4),
        "cotacao_euro_brl": round(cotacoes_para_brl["EUR"], 4),
        "horario_cotacao": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

print("🔧 FERRAMENTA 4: converter_moeda")
print(f"   Descrição: {converter_moeda.description}")

🔧 FERRAMENTA 4: converter_moeda
   Descrição: Converte valores entre moedas usando cotações atualizadas.
Moedas suportadas: USD (Dólar), EUR (Euro), BRL (Real).


In [15]:
# Testando as ferramentas individualmente

print("🧪 TESTE DAS FERRAMENTAS")
print("=" * 60)

# Teste 1: Cotação
print("\n1️⃣ Teste consultar_cotacao:")
resultado = consultar_cotacao.invoke({"simbolo": "PETR4"})
print(f"   {resultado}")

# Teste 2: Cliente
print("\n2️⃣ Teste consultar_cliente:")
resultado = consultar_cliente.invoke({"cpf": "12345678900"})
print(f"   {resultado}")

# Teste 3: Risco
print("\n3️⃣ Teste calcular_risco:")
resultado = calcular_risco.invoke({
    "valor_operacao": 50000,
    "prazo_meses": 24,
    "score_cliente": 750,
    "comprometimento_atual": 0.25
})
print(f"   {resultado}")

# Teste 4: Conversão
print("\n4️⃣ Teste converter_moeda:")
resultado = converter_moeda.invoke({
    "valor": 1000,
    "moeda_origem": "USD",
    "moeda_destino": "BRL"
})
print(f"   {resultado}")

🧪 TESTE DAS FERRAMENTAS

1️⃣ Teste consultar_cotacao:
   {'simbolo': 'PETR4', 'nome': 'Petrobras PN', 'preco_atual': 38.16, 'moeda': 'BRL', 'variacao_dia': -0.88, 'horario_consulta': '2026-01-26 23:14:41', 'status': 'mercado_fechado'}

2️⃣ Teste consultar_cliente:
   {'nome': 'Maria Silva', 'score_credito': 820, 'renda_mensal': 15000.0, 'comprometimento_renda': 0.25, 'tempo_conta_anos': 8, 'historico_atrasos': 0, 'perfil_investidor': 'Moderado', 'cpf': '12345678900', 'data_consulta': '2026-01-26 23:14:41'}

3️⃣ Teste calcular_risco:
   {'score_risco': 30, 'classificacao': 'MÉDIO', 'recomendacao': 'Aprovar com cautela - Considerar garantias adicionais', 'taxa_juros_mensal_sugerida': 1.8, 'valor_operacao': 50000.0, 'prazo_meses': 24, 'fatores_analisados': ['Score bom: +15 pontos de risco', 'Comprometimento baixo (<=30%): +5 pontos de risco', 'Prazo médio (12-36 meses): +10 pontos de risco']}

4️⃣ Teste converter_moeda:
   {'valor_original': 1000.0, 'moeda_origem': 'USD', 'valor_convertid

---

## 📚 Parte 4: O Padrão ReAct (Reasoning + Acting)

### O que é ReAct?

ReAct é um padrão de arquitetura que combina:
- **Reasoning (Raciocínio):** O agente "pensa" sobre o que precisa fazer
- **Acting (Ação):** O agente executa ferramentas para obter informações

### Ciclo ReAct

```
┌─────────────────────────────────────────────────────────────┐
│                     CICLO ReAct                              │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│   1. THOUGHT (Pensamento)                                    │
│      "Preciso descobrir a cotação atual da PETR4..."        │
│                     ↓                                        │
│   2. ACTION (Ação)                                          │
│      Chama: consultar_cotacao(simbolo="PETR4")              │
│                     ↓                                        │
│   3. OBSERVATION (Observação)                               │
│      Resultado: {"preco": 38.50, "variacao": -1.2%}         │
│                     ↓                                        │
│   4. THOUGHT (Novo pensamento)                              │
│      "Agora tenho os dados, posso formular a resposta..."   │
│                     ↓                                        │
│   5. FINAL ANSWER (Resposta Final)                          │
│      "A PETR4 está cotada a R$ 38,50, com queda de 1,2%"    │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

### Vantagens do ReAct

1. **Transparência:** Podemos ver o raciocínio do agente
2. **Debugging:** Fácil identificar onde o agente errou
3. **Flexibilidade:** O agente adapta sua estratégia com base nos resultados

---

### 💻 Implementando um Agente ReAct

In [41]:
# Criando o Agente ReAct com LangGraph
# Nota: Apesar do aviso de deprecação, esta é a API estável atual

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, SystemMessage
import warnings

# Suprime o aviso de deprecação (a nova API ainda não está disponível)
warnings.filterwarnings("ignore", category=DeprecationWarning, module="langgraph")

# Lista de ferramentas disponíveis para o agente
ferramentas = [consultar_cotacao, consultar_cliente, calcular_risco, converter_moeda]

# Configuração do LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0  # Determinístico para decisões financeiras
)

# Prompt do sistema que define o comportamento do agente
system_prompt = """Você é um Analista Financeiro Júnior altamente qualificado, especializado em análise de mercado e crédito.

## Suas Responsabilidades:
1. Analisar cotações de ações brasileiras
2. Consultar dados de clientes para análise de crédito
3. Calcular riscos de operações financeiras
4. Realizar conversões de moedas

## Diretrizes de Comportamento:
- SEMPRE use as ferramentas disponíveis quando precisar de dados atualizados
- NÃO invente dados ou cotações - use apenas informações obtidas via ferramentas
- Seja preciso e profissional em suas análises
- Quando fizer recomendações, justifique com dados concretos
- Se não tiver certeza sobre algo, diga claramente

## Formato de Resposta:
- Seja conciso mas completo
- Use formatação clara (bullets, números)
- Sempre inclua os dados que basearam sua análise
"""

# Cria o agente usando LangGraph
agente = create_react_agent(llm, ferramentas, prompt=system_prompt)

# Classe executor para interface simples compatível com o resto do notebook
class AgentExecutor:
    """Executor para o agente ReAct do LangGraph"""
    def __init__(self, agent, tools, verbose=True):
        self.agent = agent
        self.tools = tools
        self.verbose = verbose
    
    def invoke(self, inputs):
        """Invoca o agente com a entrada fornecida"""
        if self.verbose:
            print(f"🔍 Pergunta: {inputs['input']}\n")
        
        # Executa o agente
        resultado = self.agent.invoke({
            "messages": [HumanMessage(content=inputs["input"])]
        })
        
        # Mostra as ferramentas utilizadas se verbose
        if self.verbose:
            for msg in resultado["messages"]:
                if hasattr(msg, 'tool_calls') and msg.tool_calls:
                    for tc in msg.tool_calls:
                        print(f"🔧 Ferramenta usada: {tc['name']}")
        
        # Retorna a última mensagem como output
        return {"output": resultado["messages"][-1].content}

# Instancia o executor do agente
executor = AgentExecutor(
    agent=agente,
    tools=ferramentas,
    verbose=True  # Mostra o raciocínio do agente
)

print("🤖 AGENTE ANALISTA JÚNIOR configurado!")
print(f"   Ferramentas disponíveis: {[f.name for f in ferramentas]}")
print(f"   Modelo: gpt-4o-mini")
print(f"   Modo verbose: Ativado (você verá o raciocínio do agente)")

🤖 AGENTE ANALISTA JÚNIOR configurado!
   Ferramentas disponíveis: ['consultar_cotacao', 'consultar_cliente', 'calcular_risco', 'converter_moeda']
   Modelo: gpt-4o-mini
   Modo verbose: Ativado (você verá o raciocínio do agente)


C:\Users\ioann\AppData\Local\Temp\ipykernel_44084\993155982.py:44: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agente = create_react_agent(llm, ferramentas, prompt=system_prompt)


---

## 🏗️ Parte 5: Projeto Prático - O Analista Júnior

Agora vamos testar nosso agente em cenários reais do setor financeiro!

### Cenário 1: Consulta Simples de Cotação

In [39]:
# CENÁRIO 1: Consulta simples de cotação
# O agente deve identificar que precisa usar a ferramenta de cotação

print("📊 CENÁRIO 1: Consulta de Cotação")
print("=" * 60)

resposta = executor.invoke({
    "input": "Qual é a cotação atual da empresa de varejo?"
})

print("\n" + "=" * 60)
print("📝 RESPOSTA FINAL:")
print(resposta["output"])

📊 CENÁRIO 1: Consulta de Cotação
🔍 Pergunta: Qual é a cotação atual da empresa de varejo?


📝 RESPOSTA FINAL:
Para fornecer a cotação atual de uma empresa de varejo, preciso saber o símbolo da ação que você está interessado. Algumas das principais empresas de varejo no Brasil incluem:

- Magazine Luiza (MGLU3)
- Lojas Americanas (LAME4)
- Via Varejo (VVAR3)

Por favor, informe o símbolo da ação que você deseja consultar.


In [24]:
# CENÁRIO 2: Análise comparativa de múltiplas ações
# O agente deve fazer múltiplas consultas e comparar

print("📊 CENÁRIO 2: Análise Comparativa")
print("=" * 60)

resposta = executor.invoke({
    "input": "Compare as cotações de PETR4, VALE3 e ITUB4. Qual está com melhor desempenho hoje?"
})

print("\n" + "=" * 60)
print("📝 RESPOSTA FINAL:")
print(resposta["output"])

📊 CENÁRIO 2: Análise Comparativa
🔍 Pergunta: Compare as cotações de PETR4, VALE3 e ITUB4. Qual está com melhor desempenho hoje?

🔧 Ferramenta usada: consultar_cotacao
🔧 Ferramenta usada: consultar_cotacao
🔧 Ferramenta usada: consultar_cotacao

📝 RESPOSTA FINAL:
Aqui estão as cotações e desempenhos das ações PETR4, VALE3 e ITUB4:

1. **PETR4 (Petrobras PN)**
   - **Preço Atual:** R$ 39,67
   - **Variação do Dia:** +3,04%

2. **VALE3 (Vale ON)**
   - **Preço Atual:** R$ 61,16
   - **Variação do Dia:** -1,83%

3. **ITUB4 (Itaú Unibanco PN)**
   - **Preço Atual:** R$ 31,58
   - **Variação do Dia:** -3,71%

### Análise de Desempenho:
- **Melhor Desempenho:** PETR4, com uma variação positiva de 3,04%.
- **Pior Desempenho:** ITUB4, com uma variação negativa de 3,71%.

Portanto, a ação que está com melhor desempenho hoje é a **PETR4**.


In [28]:
# CENÁRIO 3: Análise de Crédito Completa
# O agente deve buscar dados do cliente e calcular o risco

print("📊 CENÁRIO 3: Análise de Crédito")
print("=" * 60)

resposta = executor.invoke({
    "input": """
    O cliente com CPF 12345678900 está solicitando um empréstimo de R$ 50.000 
    em 24 parcelas. Por favor:
    1. Consulte os dados do cliente
    2. Calcule o risco da operação
    3. Dê sua recomendação final
    """
})

print("\n" + "=" * 60)
print("📝 RESPOSTA FINAL:")
print(resposta["output"])

📊 CENÁRIO 3: Análise de Crédito
🔍 Pergunta: 
    O cliente com CPF 12345678900 está solicitando um empréstimo de R$ 50.000 
    em 24 parcelas. Por favor:
    1. Consulte os dados do cliente
    2. Calcule o risco da operação
    3. Dê sua recomendação final
    

🔧 Ferramenta usada: consultar_cliente
🔧 Ferramenta usada: calcular_risco

📝 RESPOSTA FINAL:
### Análise do Empréstimo para Maria Silva

1. **Dados do Cliente:**
   - **Nome:** Maria Silva
   - **Score de Crédito:** 820 (Excelente)
   - **Renda Mensal:** R$ 15.000
   - **Comprometimento da Renda:** 25% (R$ 3.750)
   - **Tempo de Conta:** 8 anos
   - **Histórico de Atrasos:** 0

2. **Cálculo do Risco da Operação:**
   - **Valor do Empréstimo:** R$ 50.000
   - **Prazo:** 24 meses
   - **Classificação de Risco:** BAIXO
   - **Recomendação:** Aprovar - Operação de baixo risco
   - **Taxa de Juros Mensal Sugerida:** 1.2%
   - **Fatores Analisados:**
     - Score excelente: +5 pontos de risco
     - Comprometimento baixo (<=30%): +5

In [30]:
# CENÁRIO 4: Análise de cliente de alto risco
# O agente deve identificar os problemas

print("📊 CENÁRIO 4: Cliente de Alto Risco")
print("=" * 60)

resposta = executor.invoke({
    "input": """
    Analise o cliente CPF 98765432100 para um empréstimo de R$ 30.000 em 48 meses.
    Faça uma análise completa e me diga se devemos aprovar.
    """
})

print("\n" + "=" * 60)
print("📝 RESPOSTA FINAL:")
print(resposta["output"])

📊 CENÁRIO 4: Cliente de Alto Risco
🔍 Pergunta: 
    Analise o cliente CPF 98765432100 para um empréstimo de R$ 30.000 em 48 meses.
    Faça uma análise completa e me diga se devemos aprovar.
    

🔧 Ferramenta usada: consultar_cliente
🔧 Ferramenta usada: calcular_risco

📝 RESPOSTA FINAL:
### Análise do Cliente: João Santos (CPF: 98765432100)

#### Dados do Cliente:
- **Score de Crédito:** 650
- **Renda Mensal:** R$ 5.500,00
- **Comprometimento da Renda:** 45% (R$ 2.475,00)
- **Tempo de Conta:** 2 anos
- **Histórico de Atrasos:** 3 ocorrências
- **Perfil do Investidor:** Conservador

#### Proposta de Empréstimo:
- **Valor do Empréstimo:** R$ 30.000,00
- **Prazo:** 48 meses

#### Análise de Risco:
- **Classificação de Risco:** ALTO
- **Score de Risco:** 75
- **Recomendação:** Revisar - Necessita aprovação de comitê
- **Taxa de Juros Mensal Sugerida:** 2.5%

#### Fatores Analisados:
1. **Score Médio:** +35 pontos de risco (Score de crédito abaixo de 700)
2. **Comprometimento Médio (30-50%

In [31]:
# CENÁRIO 5: Consulta mista - Cotação + Conversão
# O agente precisa combinar múltiplas ferramentas

print("📊 CENÁRIO 5: Análise com Conversão de Moeda")
print("=" * 60)

resposta = executor.invoke({
    "input": """
    Um investidor americano quer comprar 1000 ações da VALE3.
    1. Qual seria o custo em reais?
    2. Converta esse valor para dólares.
    3. Vale a pena considerando a cotação atual?
    """
})

print("\n" + "=" * 60)
print("📝 RESPOSTA FINAL:")
print(resposta["output"])

📊 CENÁRIO 5: Análise com Conversão de Moeda
🔍 Pergunta: 
    Um investidor americano quer comprar 1000 ações da VALE3.
    1. Qual seria o custo em reais?
    2. Converta esse valor para dólares.
    3. Vale a pena considerando a cotação atual?
    

🔧 Ferramenta usada: consultar_cotacao
🔧 Ferramenta usada: converter_moeda

📝 RESPOSTA FINAL:
Aqui estão as informações solicitadas sobre a compra de 1000 ações da VALE3:

1. **Custo em Reais**:
   - Preço atual da VALE3: R$ 62,93
   - Custo total para 1000 ações: 
     \[
     1000 \times 62,93 = R\$ 62.930,00
     \]

2. **Valor convertido para Dólares**:
   - Valor em Reais: R$ 62.930,00
   - Valor em Dólares: 
     \[
     R\$ 62.930,00 \approx \$ 11.350,59
     \]
   - Taxa de conversão: 1 USD = 5,5442 BRL

3. **Vale a pena considerar a cotação atual?**:
   - Para determinar se vale a pena, é importante considerar a performance da ação e as expectativas futuras. A VALE3 teve uma variação de 1,01% no dia da consulta, o que pode indicar 

In [32]:
# CENÁRIO 6: Pergunta que NÃO requer ferramentas
# O agente deve responder diretamente sem usar ferramentas

print("📊 CENÁRIO 6: Pergunta Conceitual")
print("=" * 60)

resposta = executor.invoke({
    "input": "O que é diversificação de carteira e por que é importante?"
})

print("\n" + "=" * 60)
print("📝 RESPOSTA FINAL:")
print(resposta["output"])

📊 CENÁRIO 6: Pergunta Conceitual
🔍 Pergunta: O que é diversificação de carteira e por que é importante?


📝 RESPOSTA FINAL:
A diversificação de carteira é uma estratégia de investimento que consiste em distribuir os recursos financeiros entre diferentes ativos, como ações, títulos, imóveis e outros instrumentos financeiros. O objetivo principal é reduzir o risco total da carteira, uma vez que diferentes ativos podem reagir de maneira distinta a eventos de mercado.

### Importância da Diversificação:

1. **Redução de Risco**:
   - Ao investir em uma variedade de ativos, o impacto negativo de um ativo específico na carteira é minimizado. Se um ativo tiver um desempenho ruim, outros podem compensar essa perda.

2. **Aumento do Potencial de Retorno**:
   - A diversificação permite que o investidor aproveite diferentes oportunidades de crescimento em vários setores e mercados, potencialmente aumentando o retorno total da carteira.

3. **Proteção contra Volatilidade**:
   - Mercados financei

---

## 📚 Parte 6: Adicionando Memória ao Agente

Até agora, nosso agente não lembra de conversas anteriores. Vamos adicionar memória para que ele mantenha contexto!

### Por que a Memória é Importante?

Em um cenário real de atendimento:
- Cliente menciona seu nome no início
- Faz várias perguntas relacionadas
- Espera que o agente lembre do contexto anterior

Sem memória, cada interação seria isolada.

In [ ]:
# Criando o Agente com Memória usando LangGraph
# O LangGraph tem suporte nativo a memória através do checkpointer

from langgraph.checkpoint.memory import MemorySaver

# Cria o checkpointer para memória
memory = MemorySaver()

# Recria o agente com suporte a memória
agente_com_checkpointer = create_react_agent(
    llm, 
    ferramentas, 
    prompt=system_prompt,
    checkpointer=memory  # Adiciona persistência de memória
)

# Armazenamento para histórico (para visualização)
store = {}

def get_session_history(session_id: str):
    """Retorna o histórico de mensagens para uma sessão específica."""
    if session_id not in store:
        store[session_id] = []
    return store[session_id]

# Classe executor com memória
class AgentExecutorComMemoria:
    """Executor para o agente ReAct do LangGraph com memória"""
    def __init__(self, agent, tools, verbose=True):
        self.agent = agent
        self.tools = tools
        self.verbose = verbose
    
    def invoke(self, inputs, config=None):
        """Invoca o agente com a entrada fornecida"""
        if self.verbose:
            print(f"🔍 Pergunta: {inputs['input']}\n")
        
        # Configuração do thread para memória
        thread_config = config or {"configurable": {"thread_id": "default"}}
        
        # Executa o agente
        resultado = self.agent.invoke(
            {"messages": [HumanMessage(content=inputs["input"])]},
            config=thread_config  # type: ignore
        )
        
        # Mostra as ferramentas utilizadas se verbose
        if self.verbose:
            for msg in resultado["messages"]:
                if hasattr(msg, 'tool_calls') and msg.tool_calls:
                    for tc in msg.tool_calls:
                        print(f"🔧 Ferramenta usada: {tc['name']}")
        
        # Armazena no histórico para visualização
        session_id = thread_config.get("configurable", {}).get("thread_id", "default")
        if session_id not in store:
            store[session_id] = []
        store[session_id] = resultado["messages"]
        
        # Retorna a última mensagem como output
        return {"output": resultado["messages"][-1].content}

# Instancia o executor com memória
agente_com_memoria = AgentExecutorComMemoria(
    agent=agente_com_checkpointer,
    tools=ferramentas,
    verbose=True
)

print("🧠 AGENTE COM MEMÓRIA configurado!")
print("   Agora o agente lembrará de conversas anteriores dentro da mesma sessão.")
print("   Usando LangGraph MemorySaver para persistência de estado.")

🧠 AGENTE COM MEMÓRIA configurado!
   Agora o agente lembrará de conversas anteriores dentro da mesma sessão.
   Usando LangGraph MemorySaver para persistência de estado.


C:\Users\ioann\AppData\Local\Temp\ipykernel_44084\2449423109.py:10: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agente_com_checkpointer = create_react_agent(


In [ ]:
# Demonstração da Memória - Conversa com contexto

session_id = "cliente_maria_123"
config = {"configurable": {"thread_id": session_id}}

print("💬 DEMONSTRAÇÃO: Conversa com Memória")
print("=" * 60)

# Mensagem 1
print("\n👤 USUÁRIO: Olá, meu nome é Maria e meu CPF é 12345678900. Pode verificar meus dados?")
resposta1 = agente_com_memoria.invoke(
    {"input": "Olá, meu nome é Maria e meu CPF é 12345678900. Pode verificar meus dados?"},
    config=config
)
print(f"\n🤖 AGENTE: {resposta1['output']}")

💬 DEMONSTRAÇÃO: Conversa com Memória

👤 USUÁRIO: Olá, meu nome é Maria e meu CPF é 12345678900. Pode verificar meus dados?
🔍 Pergunta: Olá, meu nome é Maria e meu CPF é 12345678900. Pode verificar meus dados?

🔧 Ferramenta usada: consultar_cliente

🤖 AGENTE: Aqui estão os dados financeiros que consegui consultar para você, Maria:

- **Nome:** Maria Silva
- **Score de Crédito:** 820 (considerado bom)
- **Renda Mensal:** R$ 15.000,00
- **Percentual de Comprometimento da Renda:** 25% (ou 0,25)
- **Tempo de Conta:** 8 anos
- **Histórico de Atrasos:** 0 (sem atrasos)
- **Perfil de Investidor:** Moderado

Se precisar de mais alguma informação ou análise, estou à disposição!


In [37]:
# Mensagem 2 - O agente deve lembrar quem é Maria
print("\n" + "=" * 60)
print("\n👤 USUÁRIO: Quero solicitar um empréstimo de R$ 80.000 em 36 meses. Qual seria o risco?")
resposta2 = agente_com_memoria.invoke(
    {"input": "Quero solicitar um empréstimo de R$ 80.000 em 36 meses. Qual seria o risco?"},
    config=config
)
print(f"\n🤖 AGENTE: {resposta2['output']}")



👤 USUÁRIO: Quero solicitar um empréstimo de R$ 80.000 em 36 meses. Qual seria o risco?
🔍 Pergunta: Quero solicitar um empréstimo de R$ 80.000 em 36 meses. Qual seria o risco?


🤖 AGENTE: Para calcular o risco da operação de crédito, preciso das seguintes informações:

1. **Score de crédito do cliente** (0-1000)
2. **Percentual atual de comprometimento da renda** (0.0 a 1.0)

Por favor, forneça esses dados para que eu possa prosseguir com a análise.


In [ ]:
# Mensagem 3 - Pergunta de follow-up
print("\n" + "=" * 60)
print("\n👤 USUÁRIO: E se eu reduzir o prazo para 24 meses, melhora alguma coisa?")
resposta3 = agente_com_memoria.invoke(
    {"input": "E se eu reduzir o prazo para 24 meses, melhora alguma coisa?"},
    config=config
)
print(f"\n🤖 AGENTE: {resposta3['output']}")

In [ ]:
# Verificando o histórico armazenado
print("\n" + "=" * 60)
print("📜 HISTÓRICO DA SESSÃO:")
print("=" * 60)

historico = get_session_history(session_id)
for i, msg in enumerate(historico):
    if hasattr(msg, 'type'):
        tipo = "👤 Human" if msg.type == "human" else "🤖 AI" if msg.type == "ai" else "🔧 Tool"
    else:
        tipo = "📝 Msg"
    conteudo = str(msg.content)[:200] + "..." if len(str(msg.content)) > 200 else str(msg.content)
    print(f"\n{i+1}. {tipo}:")
    print(f"   {conteudo}")

---

## 📚 Parte 7: Criando o Agente Final - Versão Completa

Vamos consolidar tudo em uma classe reutilizável que você pode usar em outros projetos!

In [ ]:
# Classe completa do Analista Financeiro
# Usando LangGraph para compatibilidade com versões recentes

from typing import List, Dict, Any, Optional
from langchain_openai import ChatOpenAI
from langchain_core.tools import BaseTool
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from pydantic import BaseModel, Field


class AnalistaFinanceiroConfig(BaseModel):
    """Configuração do Analista Financeiro."""
    model: str = Field(default="gpt-4o-mini", description="Modelo LLM a ser usado")
    temperature: float = Field(default=0, ge=0, le=2, description="Temperature do modelo")
    max_tokens: int = Field(default=2000, description="Máximo de tokens na resposta")
    verbose: bool = Field(default=True, description="Mostrar raciocínio do agente")
    max_iterations: int = Field(default=10, description="Máximo de iterações")


class AnalistaFinanceiro:
    """
    Agente Analista Financeiro com ferramentas de mercado e crédito.
    
    Capacidades:
    - Consulta de cotações de ações
    - Análise de dados de clientes
    - Cálculo de risco de operações
    - Conversão de moedas
    - Memória de conversas
    """
    
    def __init__(self, config: Optional[AnalistaFinanceiroConfig] = None):
        self.config = config or AnalistaFinanceiroConfig()
        self._session_store: Dict[str, List] = {}
        self._setup_agent()
    
    def _get_system_prompt(self) -> str:
        """Retorna o prompt do sistema."""
        return """
Você é um Analista Financeiro Sênior, especializado em análise de mercado e crédito do setor bancário brasileiro.

## Suas Competências:
1. **Análise de Mercado**: Consultar e interpretar cotações de ações da B3
2. **Análise de Crédito**: Avaliar perfil de clientes e calcular riscos de operações
3. **Câmbio**: Realizar conversões entre moedas com cotações atualizadas
4. **Consultoria**: Fornecer recomendações embasadas em dados

## Diretrizes Operacionais:
- SEMPRE utilize as ferramentas disponíveis para obter dados atualizados
- NUNCA invente dados financeiros - use apenas informações verificadas via ferramentas
- Seja PRECISO e PROFISSIONAL em todas as análises
- JUSTIFIQUE recomendações com dados concretos obtidos
- Se houver INCERTEZA, comunique claramente as limitações

## Padrão de Resposta:
- Estruture respostas de forma clara e organizada
- Use formatação (bullets, números) quando apropriado
- Inclua sempre os dados que fundamentam sua análise
- Para análises de risco, apresente: dados do cliente, fatores de risco, classificação e recomendação

## Compliance:
- Respeite a privacidade de dados dos clientes
- Não faça recomendações de investimento pessoal
- Indique quando uma decisão requer aprovação humana
"""
    
    def _setup_tools(self) -> List[BaseTool]:
        """Configura as ferramentas do agente."""
        return [consultar_cotacao, consultar_cliente, calcular_risco, converter_moeda]
    
    def _setup_agent(self):
        """Configura o agente usando LangGraph."""
        # LLM
        self.llm = ChatOpenAI(
            model=self.config.model,
            temperature=self.config.temperature,
            max_tokens=self.config.max_tokens  # type: ignore
        )
        
        # Tools
        self.tools = self._setup_tools()
        
        # Memory Saver para persistência
        self._memory = MemorySaver()
        
        # Cria o agente com LangGraph
        self._agent = create_react_agent(
            self.llm, 
            self.tools, 
            prompt=self._get_system_prompt(),
            checkpointer=self._memory
        )
    
    def chat(self, message: str, session_id: str = "default") -> str:
        """
        Envia uma mensagem para o agente e retorna a resposta.
        
        Args:
            message: Mensagem do usuário
            session_id: Identificador da sessão (para manter contexto)
            
        Returns:
            Resposta do agente
        """
        config = {"configurable": {"thread_id": session_id}}
        
        if self.config.verbose:
            print(f"🔍 Pergunta: {message}\n")
        
        resultado = self._agent.invoke(
            {"messages": [HumanMessage(content=message)]},
            config=config  # type: ignore
        )
        
        # Mostra ferramentas usadas se verbose
        if self.config.verbose:
            for msg in resultado["messages"]:
                if hasattr(msg, 'tool_calls') and msg.tool_calls:
                    for tc in msg.tool_calls:
                        print(f"🔧 Ferramenta usada: {tc['name']}")
        
        # Armazena histórico para visualização
        if session_id not in self._session_store:
            self._session_store[session_id] = []
        self._session_store[session_id] = resultado["messages"]
        
        return resultado["messages"][-1].content
    
    def chat_sem_memoria(self, message: str) -> str:
        """
        Envia uma mensagem sem usar memória (cada chamada é independente).
        
        Args:
            message: Mensagem do usuário
            
        Returns:
            Resposta do agente
        """
        # Cria agente temporário sem checkpointer
        agente_temp = create_react_agent(
            self.llm, 
            self.tools, 
            prompt=self._get_system_prompt()
        )
        
        if self.config.verbose:
            print(f"🔍 Pergunta: {message}\n")
        
        resultado = agente_temp.invoke(
            {"messages": [HumanMessage(content=message)]}
        )
        
        if self.config.verbose:
            for msg in resultado["messages"]:
                if hasattr(msg, 'tool_calls') and msg.tool_calls:
                    for tc in msg.tool_calls:
                        print(f"🔧 Ferramenta usada: {tc['name']}")
        
        return resultado["messages"][-1].content
    
    def limpar_sessao(self, session_id: str):
        """Limpa o histórico de uma sessão específica."""
        if session_id in self._session_store:
            del self._session_store[session_id]
            print(f"✅ Sessão '{session_id}' limpa com sucesso.")
        else:
            print(f"⚠️ Sessão '{session_id}' não encontrada.")
    
    def listar_sessoes(self) -> List[str]:
        """Lista todas as sessões ativas."""
        return list(self._session_store.keys())


print("✅ Classe AnalistaFinanceiro definida com sucesso!")

✅ Classe AnalistaFinanceiro definida com sucesso!


In [47]:
# Instanciando e testando o Analista Financeiro

# Criar instância com configuração padrão
analista = AnalistaFinanceiro(
    config=AnalistaFinanceiroConfig(
        model="gpt-4o-mini",
        temperature=0,
        verbose=True  # Vamos ver o raciocínio do agente
    )
)

print("🏦 ANALISTA FINANCEIRO v1.0")
print("=" * 60)
print(f"Modelo: {analista.config.model}")
print(f"Temperature: {analista.config.temperature}")
print(f"Ferramentas: {[t.name for t in analista.tools]}")
print("=" * 60)

🏦 ANALISTA FINANCEIRO v1.0
Modelo: gpt-4o-mini
Temperature: 0.0
Ferramentas: ['consultar_cotacao', 'consultar_cliente', 'calcular_risco', 'converter_moeda']


C:\Users\ioann\AppData\Local\Temp\ipykernel_44084\2221281433.py:89: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  self._agent = create_react_agent(


In [48]:
# Teste final: Cenário complexo de atendimento

print("\n🎯 TESTE FINAL: Atendimento Completo")
print("=" * 60)

# Simulando um atendimento completo
session = "atendimento_final_001"

# Interação 1
print("\n📞 Cliente liga para o banco...")
print("\n👤 Cliente: Boa tarde! Sou a Ana Oliveira, CPF 111.222.333-44")

resposta = analista.chat(
    "Boa tarde! Sou a Ana Oliveira, CPF 111.222.333-44. Pode me mostrar meus dados cadastrais?",
    session_id=session
)
print(f"\n🤖 Analista: {resposta}")


🎯 TESTE FINAL: Atendimento Completo

📞 Cliente liga para o banco...

👤 Cliente: Boa tarde! Sou a Ana Oliveira, CPF 111.222.333-44
🔍 Pergunta: Boa tarde! Sou a Ana Oliveira, CPF 111.222.333-44. Pode me mostrar meus dados cadastrais?

🔧 Ferramenta usada: consultar_cliente

🤖 Analista: Boa tarde, Ana Oliveira! Aqui estão os seus dados cadastrais:

- **Nome**: Ana Oliveira
- **CPF**: 111.222.333-44
- **Score de Crédito**: 750
- **Renda Mensal**: R$ 25.000,00
- **Comprometimento da Renda**: 15%
- **Tempo de Conta**: 12 anos
- **Histórico de Atrasos**: 1
- **Perfil de Investidor**: Arrojado
- **Data da Consulta**: 27 de janeiro de 2026

Se precisar de mais informações ou de alguma análise específica, estou à disposição!


In [ ]:
# Interação 2
print("\n" + "="*60)
print("\n👤 Cliente: Estou pensando em investir R$ 100.000 em ações. O que você me recomenda entre PETR4, VALE3 e WEGE3?")

resposta = analista.chat(
    "Estou pensando em investir R$ 100.000 em ações. O que você me recomenda entre PETR4, VALE3 e WEGE3?",
    session_id=session
)
print(f"\n🤖 Analista: {resposta}")

In [ ]:
# Interação 3
print("\n" + "="*60)
print("\n👤 Cliente: E quanto isso seria em dólares? Quero ter uma referência internacional.")

resposta = analista.chat(
    "E quanto isso seria em dólares? Quero ter uma referência internacional do valor total.",
    session_id=session
)
print(f"\n🤖 Analista: {resposta}")

---

## 📝 Resumo da Aula 1

### O que aprendemos:

1. **Evolução de Chains para Agentes**
   - Chains são lineares e previsíveis, mas limitadas
   - Agentes são autônomos e podem tomar decisões dinâmicas

2. **Parâmetros de Modelos LLM**
   - `temperature`: Controla criatividade (0 = determinístico)
   - `top_p` e `top_k`: Controlam diversidade de tokens
   - `tokens`: Unidades de texto que impactam custo

3. **Anatomia de um Agente**
   - 🧠 **Cérebro (LLM)**: Processa e decide
   - 💾 **Memória**: Mantém contexto de conversas
   - 🔧 **Ferramentas**: Executam ações externas

4. **Padrão ReAct**
   - Thought → Action → Observation → (repetir) → Final Answer
   - Permite raciocínio transparente e debugging

5. **Projeto: O Analista Júnior**
   - Criamos 4 ferramentas financeiras
   - Implementamos um agente completo com memória
   - Testamos em cenários reais do setor financeiro

---

### 📚 Próxima Aula: Engenharia de Dados para IA e Compliance - RAG

Na Aula 2, você aprenderá:
- Diferença entre Chat genérico e Extração estruturada
- RAG (Retrieval-Augmented Generation) para auditoria
- Técnicas de Function Calling para estruturação de saídas
- Projeto: O Auditor de Contratos

---

## 🎯 Desafio (Opcional)

Se quiser praticar antes da próxima aula, tente:
1. Adicionar uma nova ferramenta ao agente (ex: consulta de CDI)
2. Modificar o prompt do sistema para um comportamento diferente
3. Testar com diferentes valores de `temperature`

---

**Parabéns por completar a Aula 1!** 🎉